<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/tasty_annual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, time
import numpy as np
import math
import time
import json

from pathlib import Path

import pandas as pd

In [3]:
file_path = '/content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv'
file_exists = os.path.exists(file_path)
if file_exists is True:
  tasty_df = pd.read_csv(file_path)
  #tasty_df = tasty_df.reset_index()  # make sure indexes pair with number of rows
  if 'Status' not in tasty_df:
    tasty_df['Status'] = ''
  print("Tasty History loaded:", file_path)
else:
  print("Tasty History does not exists:", file_path)


Tasty History loaded: /content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv


In [6]:
def find_closed_transactions(symbol, instrumentType, df):
  return df.query('Action.str.contains("CLOSE")  and Symbol=="' + str(symbol) + '" and `Instrument Type`=="' + str(instrumentType) + '"', engine='python') 

def find_open_transactions(symbol, instrumentType, df):
  return df.query('Action.str.contains("OPEN")  and Symbol=="' + str(symbol) + '" and `Instrument Type`=="' + str(instrumentType) + '"', engine='python') 

In [9]:
for index, row in tasty_df.iterrows():
  tasty_df.loc[index,'Status'] = ''
  if "OPEN" in str(row['Action']):

    closed_df = find_closed_transactions(row['Symbol'],row['Instrument Type'], tasty_df)
    open_df = find_open_transactions(row['Symbol'],row['Instrument Type'], tasty_df)

    closed_quantity = np.sum(closed_df['Quantity'])
    open_quantity = np.sum(open_df['Quantity'])
    
    status = ''
    if closed_quantity > 0:
      if open_quantity == closed_quantity:
        status = 'CLOSED'
      else:
        status = 'PARTIALLY CLOSED'

    for closed_index, closed_row in closed_df.iterrows():
      tasty_df.loc[closed_index,'Status'] = status
    for open_index, open_row in open_df.iterrows():
      tasty_df.loc[open_index,'Status'] = status
    
    print(index, row['Date'], row['Action'], row['Symbol'], row['Quantity'], closed_quantity,  tasty_df.loc[index,'Status'])

tasty_df.to_csv(file_path)

9 2021-12-30T20:53:07+0100 BUY_TO_OPEN AFRM  220218P00100000 1.0 0.0 
10 2021-12-30T20:53:07+0100 SELL_TO_OPEN AFRM  220218P00105000 1.0 0.0 
11 2021-12-30T20:07:05+0100 SELL_TO_OPEN AMZN  220218P03120000 1.0 0.0 
12 2021-12-30T20:07:05+0100 BUY_TO_OPEN AMZN  220218P03100000 1.0 0.0 
13 2021-12-30T19:13:32+0100 SELL_TO_OPEN COST  220121C00570000 1.0 0.0 
14 2021-12-30T19:13:32+0100 BUY_TO_OPEN COST  220218C00570000 1.0 0.0 
23 2021-12-29T19:03:56+0100 SELL_TO_OPEN QCOM  220121C00200000 1.0 0.0 
24 2021-12-29T19:03:56+0100 BUY_TO_OPEN QCOM  220121C00195000 1.0 0.0 
25 2021-12-29T17:13:07+0100 SELL_TO_OPEN APPS  220318C00065000 1.0 0.0 
26 2021-12-29T17:13:07+0100 BUY_TO_OPEN APPS  220318C00060000 1.0 1.0 PARTIALLY CLOSED
27 2021-12-29T16:34:54+0100 BUY_TO_OPEN PENN  220121C00050000 1.0 0.0 
28 2021-12-29T16:34:54+0100 SELL_TO_OPEN PENN  220121C00045000 1.0 0.0 
41 2021-12-29T13:04:20+0100 SELL_TO_OPEN ARKK  230120P00090180 1.0 1.0 CLOSED
43 2021-12-29T13:02:40+0100 BUY_TO_OPEN ARKK  220